## Copy COVID

In [1]:
import warnings
import pandas as pd

warnings.filterwarnings("ignore")

In [2]:
df=pd.read_excel('https://labfile.oss.aliyuncs.com/courses/3023/China_world_cov.xlsx')
df['date']=df['date'].dt.date
df.head()

,date,month,countryName,countryShortCode,continent,countryFullName,provinceShortName,deadIncr,suspectedCountIncr,confirmedIncr,curedIncr
0,2020-01-20,1,中国,CHN,亚洲,China,北京,0,0,5,0
1,2020-01-21,1,中国,CHN,亚洲,China,北京,0,0,5,0
2,2020-01-22,1,中国,CHN,亚洲,China,北京,0,0,4,0
3,2020-01-23,1,中国,CHN,亚洲,China,北京,0,0,12,0
4,2020-01-24,1,中国,CHN,亚洲,China,北京,0,0,10,0


In [3]:
from pyecharts import options as opts
from pyecharts.charts import Bar
def building_bar(data,xaxis_index=0,yaxis_index=0):
    bar=Bar()
    bar.add_xaxis(list(data.index))
    bar.add_yaxis(" ",list(data),xaxis_index=xaxis_index,yaxis_index=yaxis_index,
                  label_opts=opts.LabelOpts(is_show=False))
    return bar

In [4]:
month=4
data=df
china=data.loc[data['countryName']=='中国']
china_p_confirm=china.loc[china['month']==month].groupby(['provinceShortName'])['confirmedIncr'].sum().sort_values(ascending=False)
china_p_bar=building_bar(china_p_confirm)
china_p_bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title='中国各省%d月确诊人数' % month),
    legend_opts=opts.LegendOpts(is_show=False),
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=18)),
    yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=18)),
    datazoom_opts=opts.DataZoomOpts(range_start=0,range_end=30),
)
china_p_bar.render_notebook()

In [5]:
month=4
data=df
world=data.loc[data['countryName']!='中国']
world

county_confirm=world.loc[world['month']==month].groupby(['countryName'])[
    'confirmedIncr'].sum().sort_values(ascending=False)
county_bar=building_bar(county_confirm)
county_bar.set_global_opts(
    title_opts=opts.TitleOpts(title='全球各国%d月确诊人数' % month),
    legend_opts=opts.LegendOpts(is_show=False),
    xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=18)),
    yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(font_size=18)),
    datazoom_opts=opts.DataZoomOpts(range_start=0,range_end=10),
)

county_bar.render_notebook()

In [6]:
from pyecharts.charts import Line
import matplotlib.colors as cs

def building_line(seriess, data):
    line=Line()
    colors = [value for value in cs.TABLEAU_COLORS.values()]
    line.add_xaxis(xaxis_data=data.index)
    for i, series in enumerate(seriess):
        line.add_yaxis(
            series_name=series,
            y_axis=data[series],
            symbol="emptyCircle",
            is_symbol_show=True,
            xaxis_index=0,
            yaxis_index=i,
            label_opts=opts.LabelOpts(is_show=False),
            linestyle_opts=opts.LineStyleOpts(width=3, color=colors[i]),
        )

    for j, series_y in enumerate(seriess[1:]):
        line.extend_axis(
            xaxis_data=data.index,
            yaxis=opts.AxisOpts(
                type_="value",
                name=series_y,
                offset=j*100-70,
                axisline_opts=opts.AxisLineOpts(
                    linestyle_opts=opts.LineStyleOpts(color=colors[j+1])
                ),
                name_textstyle_opts=opts.LabelOpts(),
                axislabel_opts=opts.LabelOpts(color=colors[j+1])
            )
        )
    line.set_global_opts(
        xaxis_opts=opts.AxisOpts(type_="category",),
        yaxis_opts=opts.AxisOpts(
            type_="value",
            name=seriess[0],
            axistick_opts=opts.AxisTickOpts(is_show=True),
            splitline_opts=opts.SplitLineOpts(is_show=True),
            axisline_opts=opts.AxisLineOpts(
                linestyle_opts=opts.LineStyleOpts(color=colors[0])),
            axislabel_opts=opts.LabelOpts(
                is_show=True, color=colors[0])
        )
    )
    return line

In [7]:
month=3
data=df
china=data.loc[data['countryName']=='中国']

line_china=china.loc[china['month'] == month].groupby(
    ['date'])['deadIncr','confirmedIncr','curedIncr'].sum()
line=building_line(seriess=['deadIncr','confirmedIncr',
                              'curedIncr'],data=line_china)
line.set_global_opts(
    legend_opts=opts.LegendOpts(pos_top="15%",pos_right="20%"),
    title_opts=opts.TitleOpts(
        title='中国各省%d月死亡/确诊/治愈人数' % month,pos_top="5%",pos_left="35%"),
)

line.render_notebook()

In [8]:
from pyecharts.charts import Map

def building_map(data_pair,center=None,zoom=1):
    m=Map()
    m.add(
        series_name="",
        data_pair=data_pair,
        maptype='china',
        label_opts=opts.LabelOpts(is_show=False),
        is_map_symbol_show=False,
        center=center,
        zoom=zoom,

    )
    return m

In [9]:
month=3
data=df
china=data.loc[data['countryName']=='中国']
china_p_confirm=china.loc[china['month']==month].groupby(['provinceShortName'])[
    'confirmedIncr'].sum().sort_values(ascending=False)
data_pair=[[province, value]
             for province, value in zip(china_p_confirm.index,china_p_confirm)]

visualmap_opts=opts.VisualMapOpts(
    is_calculable=True,
    pos_top="20%",
    pos_left="5%",
    min_=china_p_confirm.min(),
    max_=china_p_confirm[1]*1.1,
    range_text=["High", "Low"],
    range_color=[cs.TABLEAU_COLORS['tab:blue'],
                 cs.TABLEAU_COLORS['tab:orange'],
                 cs.TABLEAU_COLORS['tab:red']],
    textstyle_opts=opts.TextStyleOpts(color="black"),
)

m=building_map(data_pair=data_pair)

m.set_global_opts(
    title_opts=opts.TitleOpts(
        title='中国各省%d月确诊人数' % month,pos_top="5%",pos_left="35%"),
    visualmap_opts=visualmap_opts
)

m.render_notebook()

In [11]:
from pyecharts.charts import Grid

month=3

line_china = china.loc[china['month']==month].groupby(
    ['date'])['deadIncr', 'confirmedIncr','curedIncr'].sum()
line = building_line(seriess=['deadIncr','confirmedIncr',
                              'curedIncr'], data=line_china)
county_confirm=world.loc[world['month']==month].groupby(['countryName'])[
    'confirmedIncr'].sum().sort_values(ascending=False)
county_bar=building_bar(county_confirm,xaxis_index=1,yaxis_index=3)
china_p_confirm=china.loc[china['month']==month].groupby(['provinceShortName'])[
    'confirmedIncr'].sum().sort_values(ascending=False)
china_p_bar=building_bar(china_p_confirm,xaxis_index=2,yaxis_index=4)

data_pair=[[province, value]
             for province,value in zip(china_p_confirm.index,china_p_confirm)]
m=building_map(data_pair=data_pair,center=[140,20],zoom=0.6)

visualmap_opts = opts.VisualMapOpts(
    is_calculable=True,
    pos_top="20%",
    pos_left="45%",
    min_=china_p_confirm.min(),
    max_=china_p_confirm[1]*1.1,
    range_text=["High", "Low"],
    range_color=[cs.TABLEAU_COLORS['tab:blue'],
                 cs.TABLEAU_COLORS['tab:orange'],
                 cs.TABLEAU_COLORS['tab:red']],
    textstyle_opts=opts.TextStyleOpts(color="black"),
)

line.set_global_opts(
    legend_opts=opts.LegendOpts(pos_top="58%",pos_right="60%"),
    title_opts=opts.TitleOpts(
        title='中国各省%d月死亡/确诊/治愈人数' % month,pos_top="55%",pos_left="15%"),
    visualmap_opts=visualmap_opts,
    datazoom_opts=[
        opts.DataZoomOpts(
            pos_bottom="55%",
            range_start=0,
            range_end=15,
            xaxis_index=[1],
        ),
        opts.DataZoomOpts(
            pos_bottom="1%",
            range_start=0,
            range_end=40,
            xaxis_index=[2]
        ),
        opts.DataZoomOpts(
            pos_bottom="1%",
            pos_left="10%",
            range_start=0,
            range_end=300,
            xaxis_index=[0]
        )
    ],
)

county_bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title='全球各国%d月确诊人数' % month, pos_top="0%", pos_right="20%"),
    legend_opts=opts.LegendOpts(is_show=False)
)
china_p_bar.set_global_opts(
    title_opts=opts.TitleOpts(
        title='中国各省%d月确诊人数' % month, pos_top="50%", pos_right="20%"),
    legend_opts=opts.LegendOpts(is_show=False)

)
m.set_global_opts(
    title_opts=opts.TitleOpts(
        title='中国各省%d月确诊人数' % month, pos_top="1%", pos_left="20%"),

)
grid=Grid(init_opts=opts.InitOpts(width="1200px",height="800px"))
grid.add(line,is_control_axis_index=True,grid_opts=opts.GridOpts(
    pos_right="60%",pos_top="65%",pos_bottom="8%"))
grid.add(county_bar,is_control_axis_index=True,grid_opts=opts.GridOpts(
    pos_left="60%",pos_top="5%",pos_bottom="65%"))
grid.add(china_p_bar,is_control_axis_index=True,grid_opts=opts.GridOpts(
    pos_left="60%",pos_top="55%",pos_bottom="8%"))
grid.add(m, grid_opts=opts.GridOpts(
    pos_left="60%",pos_top="55%",pos_bottom="10%"))

grid.render_notebook()

### Add Month Grid

In [14]:
def get_month_grid(month):
    line_china=china.loc[china['month']==month].groupby(
        ['date'])['deadIncr','confirmedIncr','curedIncr'].sum()
    line=building_line(seriess=['deadIncr','confirmedIncr',
                                  'curedIncr'],data=line_china)

    county_confirm=world.loc[world['month']==month].groupby(['countryName'])[
        'confirmedIncr'].sum().sort_values(ascending=False)
    county_bar=building_bar(county_confirm,xaxis_index=1,yaxis_index=3)

    china_p_confirm=china.loc[china['month']==month].groupby(['provinceShortName'])[
        'confirmedIncr'].sum().sort_values(ascending=False)
    china_p_bar=building_bar(
        china_p_confirm,xaxis_index=2,yaxis_index=4)

    data_pair=[[province, value]
                 for province,value in zip(china_p_confirm.index,china_p_confirm)]
    m=building_map(data_pair=data_pair,center=[140,20],zoom=0.6)

    visualmap_opts = opts.VisualMapOpts(
        is_calculable=True,
        pos_top="20%",
        pos_left="45%",
        min_=china_p_confirm.min(),
        max_=china_p_confirm[1]*1.1,
        range_text=["High", "Low"],
        range_color=[cs.TABLEAU_COLORS['tab:blue'],
                     cs.TABLEAU_COLORS['tab:orange'],
                     cs.TABLEAU_COLORS['tab:red']],
        textstyle_opts=opts.TextStyleOpts(color="black"),
    )

    line.set_global_opts(
        legend_opts=opts.LegendOpts(pos_top="58%", pos_right="60%"),
        title_opts=opts.TitleOpts(
            title='中国各省%d月死亡/确诊/治愈人数' % month, pos_top="55%", pos_left="15%"),
        visualmap_opts=visualmap_opts,
        datazoom_opts=[
            opts.DataZoomOpts(
                pos_bottom="55%",
                range_start=0,
                range_end=20,
                xaxis_index=[1],
            ),
            opts.DataZoomOpts(
                pos_bottom="1%",
                range_start=0,
                range_end=60,
                xaxis_index=[2]
            ),
            opts.DataZoomOpts(
                pos_bottom="1%",
                pos_left="10%",
                range_start=0,
                range_end=300,
                xaxis_index=[0]
            )
        ],
    )

    county_bar.set_global_opts(
        title_opts=opts.TitleOpts(
            title='全球各国%d月确诊人数' % month, pos_top="0%", pos_right="20%"),
        legend_opts=opts.LegendOpts(is_show=False)
    )
    china_p_bar.set_global_opts(
        title_opts=opts.TitleOpts(
            title='中国各省%d月确诊人数' % month, pos_top="50%", pos_right="20%"),
        legend_opts=opts.LegendOpts(is_show=False)
    )
    m.set_global_opts(
        title_opts=opts.TitleOpts(
            title='中国各省%d月确诊人数' % month, pos_top="1%", pos_left="20%"),
    )

    grid = Grid(init_opts=opts.InitOpts(width="1200px", height="800px"))

    grid.add(line, is_control_axis_index=True, grid_opts=opts.GridOpts(
        pos_right="60%", pos_top="65%", pos_bottom="8%"))   
    grid.add(county_bar, is_control_axis_index=True, grid_opts=opts.GridOpts(
        pos_left="60%", pos_top="5%", pos_bottom="65%"))   
    grid.add(china_p_bar, is_control_axis_index=True, grid_opts=opts.GridOpts(
        pos_left="60%", pos_top="55%", pos_bottom="8%"))  
    grid.add(m, grid_opts=opts.GridOpts(
        pos_left="60%", pos_top="55%", pos_bottom="10%"))

    return grid

In [15]:
from pyecharts.charts import Timeline
import numpy as np
from pyecharts.globals import ThemeType

timeline = Timeline(
    init_opts=opts.InitOpts(
        width='1200px',
        height='750px',
        theme=ThemeType.ESSOS,
    )
)

timeline.add_schema(play_interval=1200,
                    pos_top="5%",
                    pos_left="1%",
                    pos_right="65%",
                    pos_bottom="90%",                  
                   )  

for month in np.arange(1,8,1):
    grid=get_month_grid(month)
    timeline.add(grid, "{}月".format(month))


timeline.render_notebook()